In [1]:
#Importing Libraries
# please do go through this python notebook: 
import warnings
warnings.filterwarnings("ignore")

import csv
import pandas as pd#pandas to create small dataframes 
import datetime #Convert to unix time
import time #Convert to unix time
# if numpy is not installed already : pip3 install numpy
import numpy as np#Do aritmetic operations on arrays
# matplotlib: used to plot graphs
import matplotlib
import matplotlib.pylab as plt
import seaborn as sns#Plots
from matplotlib import rcParams#Size of plots  
from sklearn.cluster import MiniBatchKMeans, KMeans#Clustering
import math
import pickle
import os
# to install xgboost: pip3 install xgboost
import xgboost as xgb

import warnings
import networkx as nx
import pdb
import pickle
from pandas import HDFStore,DataFrame
from pandas import read_hdf
from scipy.sparse.linalg import svds, eigs
import gc
from tqdm import tqdm

# Reading Data

In [2]:
train_graph=nx.read_edgelist('train.csv',delimiter=',',create_using=nx.DiGraph(),nodetype=int)
print(nx.info(train_graph))

DiGraph with 1862220 nodes and 9437519 edges


 # Similarity measures

## Jaccard Distance:
http://www.statisticshowto.com/jaccard-index/

\begin{equation}
j = \frac{|X\cap Y|}{|X \cup Y|} 
\end{equation}

In [3]:
#for followees
def jaccard_for_followees(a,b):
    try:
        if len(set(train_graph.successors(a))) == 0  | len(set(train_graph.successors(b))) == 0:
            return 0
        sim = (len(set(train_graph.successors(a)).intersection(set(train_graph.successors(b)))))/\
                                    (len(set(train_graph.successors(a)).union(set(train_graph.successors(b)))))
    except:
        return 0
    return sim

In [4]:
#for followers
def jaccard_for_followers(a,b):
    try:
        if len(set(train_graph.predecessors(a))) == 0  | len(set(g.predecessors(b))) == 0:
            return 0
        sim = (len(set(train_graph.predecessors(a)).intersection(set(train_graph.predecessors(b)))))/\
                                 (len(set(train_graph.predecessors(a)).union(set(train_graph.predecessors(b)))))
        return sim
    except:
        return 0

## Cosine distance

\begin{equation}
CosineDistance = \frac{|X\cap Y|}{|X|\cdot|Y|} 
\end{equation}

In [5]:
#for followees
def cosine_for_followees(a,b):
    try:
        if len(set(train_graph.successors(a))) == 0  | len(set(train_graph.successors(b))) == 0:
            return 0
        sim = (len(set(train_graph.successors(a)).intersection(set(train_graph.successors(b)))))/\
                                    (math.sqrt(len(set(train_graph.successors(a)))*len((set(train_graph.successors(b))))))
        return sim
    except:
        return 0

In [6]:
def cosine_for_followers(a,b):
    try:
        
        if len(set(train_graph.predecessors(a))) == 0  | len(set(train_graph.predecessors(b))) == 0:
            return 0
        sim = (len(set(train_graph.predecessors(a)).intersection(set(train_graph.predecessors(b)))))/\
                                     (math.sqrt(len(set(train_graph.predecessors(a))))*(len(set(train_graph.predecessors(b)))))
        return sim
    except:
        return 0

## Ranking Measures

https://networkx.github.io/documentation/networkx-1.10/reference/generated/networkx.algorithms.link_analysis.pagerank_alg.pagerank.html

PageRank computes a ranking of the nodes in the graph G based on the structure of the incoming links.


Mathematical PageRanks for a simple network, expressed as percentages. (Google uses a logarithmic scale.) Page C has a higher PageRank than Page E, even though there are fewer links to C; the one link to C comes from an important page and hence is of high value. If web surfers who start on a random page have an 85% likelihood of choosing a random link from the page they are currently visiting, and a 15% likelihood of jumping to a page chosen at random from the entire web, they will reach Page E 8.1% of the time. <b>(The 15% likelihood of jumping to an arbitrary page corresponds to a damping factor of 85%.) Without damping, all web surfers would eventually end up on Pages A, B, or C, and all other pages would have PageRank zero. In the presence of damping, Page A effectively links to all pages in the web, even though it has no outgoing links of its own.</b>

## Page Ranking

https://en.wikipedia.org/wiki/PageRank



In [7]:
if not os.path.isfile('page_rank.p'):
    pr = nx.pagerank(train_graph, alpha=0.85)
    pickle.dump(pr,open('page_rank.p','wb'))
else:
    pr = pickle.load(open('page_rank.p','rb'))

In [8]:
print('min',pr[min(pr, key=pr.get)])
print('max',pr[max(pr, key=pr.get)])
print('mean',float(sum(pr.values())) / len(pr))

min 1.478340267341635e-07
max 2.688578764999046e-05
mean 5.369934808940616e-07


In [9]:
#for imputing to nodes which are not there in Train data
mean_pr = float(sum(pr.values())) / len(pr)
print(mean_pr)

5.369934808940616e-07


## Shortest path:

In [10]:
#if has direct edge then deleting that edge and calculating shortest path
def compute_shortest_path_length(a,b):
    p=-1
    try:
        if train_graph.has_edge(a,b):
            train_graph.remove_edge(a,b)
            p= nx.shortest_path_length(train_graph,source=a,target=b)
            train_graph.add_edge(a,b)
        else:
            p= nx.shortest_path_length(train_graph,source=a,target=b)
        return p
    except:
        return -1

## Checking for same community

In [11]:
#getting weekly connected edges from graph 
wcc=list(nx.weakly_connected_components(train_graph))
def belongs_to_same_wcc(a,b):
    index = []
    if train_graph.has_edge(b,a):
        return 1
    if train_graph.has_edge(a,b):
            for i in wcc:
                if a in i:
                    index= i
                    break
            if (b in index):
                train_graph.remove_edge(a,b)
                if compute_shortest_path_length(a,b)==-1:
                    train_graph.add_edge(a,b)
                    return 0
                else:
                    train_graph.add_edge(a,b)
                    return 1
            else:
                return 0
    else:
            for i in wcc:
                if a in i:
                    index= i
                    break
            if(b in index):
                return 1
            else:
                return 0

## Adamic/Adar Index:
Adamic/Adar measures is defined as inverted sum of degrees of common neighbours for given two vertices.
$$A(x,y)=\sum_{u \in N(x) \cap N(y)}\frac{1}{log(|N(u)|)}$$

X is considered the source and y the destination node, N(u) are the common neighbours of the two.

In [12]:
#adar index
def calc_adar_in(a,b):
    sum=0
    try:
        n=list(set(train_graph.successors(a)).intersection(set(train_graph.successors(b))))
        if len(n)!=0:
            for i in n:
                sum=sum+(1/np.log10(len(list(train_graph.predecessors(i)))))
            return sum
        else:
            return 0
    except:
        return 0

## Following back

In [13]:
def follows_back(a,b):
    if train_graph.has_edge(b,a):
        return 1
    else:
        return 0

## Katz Centrality:

In [14]:
if not os.path.isfile('katz.p'):
    katz = nx.katz.katz_centrality(train_graph,alpha=0.005,beta=1)
    pickle.dump(katz,open('katz.p','wb'))
else:
    katz = pickle.load(open('katz.p','rb'))

In [15]:
print('min',katz[min(katz, key=katz.get)])
print('max',katz[max(katz, key=katz.get)])
print('mean',float(sum(katz.values())) / len(katz))

min 0.0007101687329270317
max 0.00557733722363664
mean 0.0007306395563933362


In [16]:
mean_katz = float(sum(katz.values())) / len(katz)
print(mean_katz)

0.0007306395563933362


## Hits Algorithm

In [17]:
if not os.path.isfile('hits.p'):
    hits = nx.hits(train_graph, max_iter=100, tol=1e-08, nstart=None, normalized=True)
    pickle.dump(hits,open('hits.p','wb'))
else:
    hits = pickle.load(open('hits.p','rb'))

In [18]:
print('min',hits[0][min(hits[0], key=hits[0].get)])
print('max',hits[0][max(hits[0], key=hits[0].get)])
print('mean',float(sum(hits[0].values())) / len(hits[0]))

min -2.249912022959974e-20
max 0.004813673061976435
mean 5.369934808962772e-07


## Choosing the number of occasions that will be trained

In [19]:
import random
if os.path.isfile('Classification.csv'):
    filename = "Classification.csv"
    # you uncomment this line, if you dont know the lentgh of the file name
    # here we have hardcoded the number of lines as 15100030
    n_train = sum(1 for line in open(filename)) #number of records in file (excludes header)
    #n_train =  15100028
    s = 100000 # HOW MANY TO READ
    skip_train = sorted(random.sample(range(1,n_train+1),n_train-s))
    #https://stackoverflow.com/a/22259008/4084039

# From Sample to Full

In [20]:
df_final_train = pd.read_csv('Classification.csv', skiprows=skip_train, names=['source', 'destination'])
#df_final_train = pd.read_csv('Classification.csv', names=['source', 'destination'])
df_final_train['connected'] = pd.read_csv('train_y.csv', skiprows=skip_train, names=['connected'])
#df_final_train['connected'] = pd.read_csv('train_y.csv', names=['connected'])

print("Our train matrix size ",df_final_train.shape)
df_final_train.head(2)

Our train matrix size  (100001, 3)


,source,destination,connected
0,1,690569,1
1,18,1003537,1


## Adding features as collumns


In [21]:
if not os.path.isfile('storage_sample_stage1.h5'):
    #mapping jaccrd followers to train and test data
    df_final_train['jaccard_followers'] = df_final_train.apply(lambda row:
                                            jaccard_for_followers(row['source'],row['destination']),axis=1)


    #mapping jaccrd followees to train and test data
    df_final_train['jaccard_followees'] = df_final_train.apply(lambda row:
                                            jaccard_for_followees(row['source'],row['destination']),axis=1)

        #mapping jaccrd followers to train and test data
    df_final_train['cosine_followers'] = df_final_train.apply(lambda row:
                                            cosine_for_followers(row['source'],row['destination']),axis=1)

    #mapping jaccrd followees to train and test data
    df_final_train['cosine_followees'] = df_final_train.apply(lambda row:
                                            cosine_for_followees(row['source'],row['destination']),axis=1)

In [22]:
def compute_features_stage1(df_final):
    #calculating no of followers followees for source and destination
    #calculating intersection of followers and followees for source and destination
    num_followers_s=[]
    num_followees_s=[]
    num_followers_d=[]
    num_followees_d=[]
    inter_followers=[]
    inter_followees=[]
    for i,row in df_final.iterrows():
        try:
            s1=set(train_graph.predecessors(row['source']))
            s2=set(train_graph.successors(row['source']))
        except:
            s1 = set()
            s2 = set()
        try:
            d1=set(train_graph.predecessors(row['destination']))
            d2=set(train_graph.successors(row['destination']))
        except:
            d1 = set()
            d2 = set()
        num_followers_s.append(len(s1))
        num_followees_s.append(len(s2))

        num_followers_d.append(len(d1))
        num_followees_d.append(len(d2))

        inter_followers.append(len(s1.intersection(d1)))
        inter_followees.append(len(s2.intersection(d2)))
    
    return num_followers_s, num_followers_d, num_followees_s, num_followees_d, inter_followers, inter_followees

In [23]:
if not os.path.isfile('storage_sample_stage1.h5'):
    df_final_train['num_followers_s'], df_final_train['num_followers_d'], \
    df_final_train['num_followees_s'], df_final_train['num_followees_d'], \
    df_final_train['inter_followers'], df_final_train['inter_followees']= compute_features_stage1(df_final_train)
    

    hdf = HDFStore('storage_sample_stage1.h5')
    hdf.put('train_df',df_final_train, format='table', data_columns=True)
    hdf.close()
else:
    df_final_train = read_hdf('storage_sample_stage1.h5', 'train_df',mode='r')

## Stage 2

In [24]:
if not os.path.isfile('storage_sample_stage2.h5'):
    #mapping adar index on train
    df_final_train['adar_index'] = df_final_train.apply(lambda row: calc_adar_in(row['source'],row['destination']),axis=1)

    #--------------------------------------------------------------------------------------------------------
    #mapping followback or not on train
    df_final_train['follows_back'] = df_final_train.apply(lambda row: follows_back(row['source'],row['destination']),axis=1)

    #--------------------------------------------------------------------------------------------------------
    #mapping same component of wcc or not on train
    df_final_train['same_comp'] = df_final_train.apply(lambda row: belongs_to_same_wcc(row['source'],row['destination']),axis=1)
    
#    --------------------------------------------------------------------------------------------------------
    #mapping shortest path on train 
    df_final_train['shortest_path'] = df_final_train.apply(lambda row: compute_shortest_path_length(row['source'],row['destination']),axis=1)
    #mapping shortest path on test

    hdf = HDFStore('storage_sample_stage2.h5')
    hdf.put('train_df',df_final_train, format='table', data_columns=True)
    hdf.close()
else:
    df_final_train = read_hdf('storage_sample_stage2.h5', 'train_df',mode='r')

## Stage 3


<li>Weight Features
    <ul>
        <li>weight of incoming edges</li>
        <li>weight of outgoing edges</li>
        <li>weight of incoming edges + weight of outgoing edges</li>
        <li>weight of incoming edges * weight of outgoing edges</li>
        <li>2*weight of incoming edges + weight of outgoing edges</li>
        <li>weight of incoming edges + 2*weight of outgoing edges</li>
    </ul>
</li>


#### Weight Features

In order to determine the similarity of nodes, an edge weight value was calculated between nodes. Edge weight decreases as the neighbor count goes up. Intuitively, consider one million people following a celebrity on a social network then chances are most of them never met each other or the celebrity. On the other hand, if a user has 30 contacts in his/her social network, the chances are higher that many of them know each other. 
`credit` - Graph-based Features for Supervised Link Prediction
William Cukierski, Benjamin Hamner, Bo Yang

\begin{equation}
W = \frac{1}{\sqrt{1+|X|}}
\end{equation}

it is directed graph so calculated Weighted in and Weighted out differently

In [25]:
#weight for source and destination of each link
Weight_in = {}
Weight_out = {}
for i in  tqdm(train_graph.nodes()):
    s1=set(train_graph.predecessors(i))
    w_in = 1.0/(np.sqrt(1+len(s1)))
    Weight_in[i]=w_in
    
    s2=set(train_graph.successors(i))
    w_out = 1.0/(np.sqrt(1+len(s2)))
    Weight_out[i]=w_out
    
#for imputing with mean
mean_weight_in = np.mean(list(Weight_in.values()))
mean_weight_out = np.mean(list(Weight_out.values()))

100%|████████████████████████████████████████████████████████████████████| 1862220/1862220 [00:13<00:00, 138227.68it/s]


In [26]:
if not os.path.isfile('storage_sample_stage3.h5'):
    #mapping to pandas train
    df_final_train['weight_in'] = df_final_train.destination.apply(lambda x: Weight_in.get(x,mean_weight_in))
    df_final_train['weight_out'] = df_final_train.source.apply(lambda x: Weight_out.get(x,mean_weight_out))


    #some features engineerings on the in and out weights
    df_final_train['weight_f1'] = df_final_train.weight_in + df_final_train.weight_out
    df_final_train['weight_f2'] = df_final_train.weight_in * df_final_train.weight_out
    df_final_train['weight_f3'] = (2*df_final_train.weight_in + 1*df_final_train.weight_out)
    df_final_train['weight_f4'] = (1*df_final_train.weight_in + 2*df_final_train.weight_out)


In [27]:
if not os.path.isfile('storage_sample_stage3.h5'):
    
    #page rank 
    #if anything not there in train graph then adding mean page rank 
    df_final_train['page_rank_s'] = df_final_train.source.apply(lambda x:pr.get(x,mean_pr))
    df_final_train['page_rank_d'] = df_final_train.destination.apply(lambda x:pr.get(x,mean_pr))


    #================================================================================

    #Katz centrality score 
    #if anything not there in train graph then adding mean katz score
    df_final_train['katz_s'] = df_final_train.source.apply(lambda x: katz.get(x,mean_katz))
    df_final_train['katz_d'] = df_final_train.destination.apply(lambda x: katz.get(x,mean_katz))


    #================================================================================

    #Hits algorithm score 
    #if anything not there in train graph then adding 0
    df_final_train['hubs_s'] = df_final_train.source.apply(lambda x: hits[0].get(x,0))
    df_final_train['hubs_d'] = df_final_train.destination.apply(lambda x: hits[0].get(x,0))

    #================================================================================

    #Hits algorithm score 
    #if anything not there in train graph then adding 0
    df_final_train['authorities_s'] = df_final_train.source.apply(lambda x: hits[1].get(x,0))
    df_final_train['authorities_d'] = df_final_train.destination.apply(lambda x: hits[1].get(x,0))


    #================================================================================

    hdf = HDFStore('storage_sample_stage3.h5')
    hdf.put('train_df',df_final_train, format='table', data_columns=True)
    hdf.close()
else:
    df_final_train = read_hdf('storage_sample_stage3.h5', 'train_df',mode='r')

In [28]:
df_final_train.head()

,source,destination,connected,jaccard_followers,jaccard_followees,cosine_followers,cosine_followees,num_followers_s,num_followers_d,num_followees_s,...,weight_f3,weight_f4,page_rank_s,page_rank_d,katz_s,katz_d,hubs_s,hubs_d,authorities_s,authorities_d
0,1,690569,1,0,0.090909,0.039817,0.251976,3,29,3,...,0.865148,1.182574,2.979445e-07,3.726469e-06,0.000722,0.000819,8.549207e-18,1.690629e-16,3.812413e-18,3.942273e-16
1,18,1003537,1,0,0.058824,0.092450,0.149071,13,3,15,...,1.250000,1.000000,1.111717e-06,3.046637e-07,0.000759,0.000721,1.583866e-17,9.329002e-20,6.343111e-18,1.297053e-19
2,25,992602,1,0,0.242424,0.138866,0.440386,21,11,30,...,0.756956,0.647886,1.249942e-06,6.286333e-07,0.000790,0.000753,1.054783e-13,6.601673e-14,6.990593e-15,3.079403e-15
3,61,1408376,1,0,0.315789,0.151523,0.514496,8,7,17,...,0.942809,0.824958,6.565506e-07,4.826817e-07,0.000740,0.000736,2.768741e-19,2.050358e-19,6.563911e-21,6.826043e-21
4,62,402932,1,0,0.000000,0.149071,0.000000,5,3,3,...,1.500000,1.500000,4.097580e-07,8.216330e-07,0.000729,0.000721,3.984634e-18,-0.000000e+00,1.731093e-16,2.732624e-19


### Adding new feature Preferential Attachement 

 One well-known concept in social networks is that users with many friends tend to create more connections in the future. This is due to the fact that in some social networks, like in finance, the rich get richer. We estimate how ”rich” our two vertices are by calculating the multiplication between the number of friends (|Γ(x)|) or followers each vertex has.

#### Preferential Attachement for followers

In [29]:
#for train dataset
nfs=np.array(df_final_train['num_followers_s'])
nfd=np.array(df_final_train['num_followers_d'])
preferential_followers=[]
for i in range(len(nfs)):
    preferential_followers.append(nfd[i]*nfs[i])
df_final_train['prefer_Attach_followers']= preferential_followers
df_final_train.head()

,source,destination,connected,jaccard_followers,jaccard_followees,cosine_followers,cosine_followees,num_followers_s,num_followers_d,num_followees_s,...,weight_f4,page_rank_s,page_rank_d,katz_s,katz_d,hubs_s,hubs_d,authorities_s,authorities_d,prefer_Attach_followers
0,1,690569,1,0,0.090909,0.039817,0.251976,3,29,3,...,1.182574,2.979445e-07,3.726469e-06,0.000722,0.000819,8.549207e-18,1.690629e-16,3.812413e-18,3.942273e-16,87
1,18,1003537,1,0,0.058824,0.092450,0.149071,13,3,15,...,1.000000,1.111717e-06,3.046637e-07,0.000759,0.000721,1.583866e-17,9.329002e-20,6.343111e-18,1.297053e-19,39
2,25,992602,1,0,0.242424,0.138866,0.440386,21,11,30,...,0.647886,1.249942e-06,6.286333e-07,0.000790,0.000753,1.054783e-13,6.601673e-14,6.990593e-15,3.079403e-15,231
3,61,1408376,1,0,0.315789,0.151523,0.514496,8,7,17,...,0.824958,6.565506e-07,4.826817e-07,0.000740,0.000736,2.768741e-19,2.050358e-19,6.563911e-21,6.826043e-21,56
4,62,402932,1,0,0.000000,0.149071,0.000000,5,3,3,...,1.500000,4.097580e-07,8.216330e-07,0.000729,0.000721,3.984634e-18,-0.000000e+00,1.731093e-16,2.732624e-19,15


#### Preferential Attachement for followees

In [30]:
#for train dataset
nfs=np.array(df_final_train['num_followees_s'])
nfd=np.array(df_final_train['num_followees_d'])
preferential_followees=[]
for i in range(len(nfs)):
    preferential_followees.append(nfd[i]*nfs[i])
df_final_train['prefer_Attach_followees']= preferential_followees
df_final_train.head()

,source,destination,connected,jaccard_followers,jaccard_followees,cosine_followers,cosine_followees,num_followers_s,num_followers_d,num_followees_s,...,page_rank_s,page_rank_d,katz_s,katz_d,hubs_s,hubs_d,authorities_s,authorities_d,prefer_Attach_followers,prefer_Attach_followees
0,1,690569,1,0,0.090909,0.039817,0.251976,3,29,3,...,2.979445e-07,3.726469e-06,0.000722,0.000819,8.549207e-18,1.690629e-16,3.812413e-18,3.942273e-16,87,63
1,18,1003537,1,0,0.058824,0.092450,0.149071,13,3,15,...,1.111717e-06,3.046637e-07,0.000759,0.000721,1.583866e-17,9.329002e-20,6.343111e-18,1.297053e-19,39,45
2,25,992602,1,0,0.242424,0.138866,0.440386,21,11,30,...,1.249942e-06,6.286333e-07,0.000790,0.000753,1.054783e-13,6.601673e-14,6.990593e-15,3.079403e-15,231,330
3,61,1408376,1,0,0.315789,0.151523,0.514496,8,7,17,...,6.565506e-07,4.826817e-07,0.000740,0.000736,2.768741e-19,2.050358e-19,6.563911e-21,6.826043e-21,56,136
4,62,402932,1,0,0.000000,0.149071,0.000000,5,3,3,...,4.097580e-07,8.216330e-07,0.000729,0.000721,3.984634e-18,-0.000000e+00,1.731093e-16,2.732624e-19,15,0


In [31]:

hdf = HDFStore('storage_sample_stage4.h5')
hdf.put('train_df',df_final_train, format='table', data_columns=True)
hdf.close()

In [32]:
df_final_train.groupby(by="connected")

In [33]:
df_final_train["connected"].value_counts()

0    50153
1    49848
Name: connected, dtype: int64